In [39]:
from core import *

<b>The whole model works in this order</b>
<ol>
    <li>clasp model - in: features ts [98][n] - out: prediction [98][m]</li>
    <li>split saliency - in: prediction[98][m] - out: pred_split [6][k][m]</li>
    <li>combinesinglesaliency - in: pred_split[6][k][m] - out: salience_preds [6][m]</li>
    <li>union salience_preds - in: salience_preds [6][m] - out: union_pred [m]</li>
    <li>ignorezone - in: union_pred [m] - out: ignored_pred [m]</li>
    <li>delnear - in: ignored_pred [m] - out: del_pred [m]</li>
    <li>DTW - in: del_pred [m] - out: dtw_pred [m]</li>
    <li>scoring - in: dtw_pred [m] - out: f1score</li>
</ol>

This project is divided into three part, because each part takes a lot of days for computing the result:
<ol>
    <li>Clasp Model: (1) -> create all prediction from the clasp model</li>
    <li>Filtering: (2)(3)(4)(5)(6)(7) -> filters all prediction to remove false positives</li>
    <li>Scoring: (8) -> given the filtered prediction, gives an evaluation</li>
</ol>

<b> Clasp Model (1)</b>
We use ParameterGrid() to compute all the combination given the param_grid and we obtained <b>19</b> different models

In [ ]:
param_grid = {
    "window_size": ['suss'],
    "distance": ['euclidean_distance'],
    "score":['roc_auc','f1'],
    "early_stopping": [True,False],
    "k_neighbours":[1,3,5,7,9],
    "n_jobs":[4]
}


# 21*98 = 2058 timeseries
for p_idx,params in enumerate(ParameterGrid(param_grid)):
    params["excl_radius"]=params["k_neighbours"]+2
    cpsl=[]
    for i in range(len(TIMESERIES)):
        df=ReadAndPreProcess(TIMESERIES[i])
        gt=LoadingGroundTruth(df,GROUNDTRUTH[i])
        #cp_all,cps,clasp=GetClasp2(df,gt,0,list(range(len(features_name))), window_size="suss",distance="euclidean_distance",n_jobs=6)
        cp_all,cps,clasp=GetClasp2(df,gt,list(range(len(features_name))),p_idx,i, **params)
        cpsl.append(cps)


    with open("../cpsl_param_"+str(p_idx)+".pkl", "wb") as f:
        pickle.dump(cpsl, f)


<b>Filtering (2)(3)(4)(5)(6)(7)</b>
The gridsearch isn't finished yet. Each step has its known set of parameters that needs to be found to have the optimal model. So in the filtering phase we gridsearch all values and generate all possible filtered prediction from each different. 

The computational costs is made by for loops:
<p>for each models = 19</p>
<p style="text-indent: 15px;">for each params = 243 </p>
<p style="text-indent: 30px;">for each videos = 21 </p>
<p style="text-indent: 45px;">for each features = 98</p>

In [30]:
with open("../dfl.pkl", "rb") as f:
    dfl = pickle.load(f)

with open("../gtl.pkl", "rb") as f:
    gtl = pickle.load(f)

In [4]:
#1. model setting: already computed with given upper hyperparameters
modelpred=[]
for i in range(0,20):
    with open("../cpsl_param_"+str(i)+".pkl", "rb") as f:
        modelpred.append(pickle.load(f))

#20 model
#21 video
#98 feature

In [2]:
param_grid = {
    "window_size": ['suss'],
    "distance": ['euclidean_distance'],
    "score":['roc_auc','f1'],
    "early_stopping": [True,False],
    "k_neighbours":[1,3,5,7,9],
    "n_jobs":[4]
}
for p_idx,params in enumerate(ParameterGrid(param_grid)):
    print(f'idx:{p_idx} {params}')

idx:0 {'distance': 'euclidean_distance', 'early_stopping': True, 'k_neighbours': 1, 'n_jobs': 4, 'score': 'roc_auc', 'window_size': 'suss'}
idx:1 {'distance': 'euclidean_distance', 'early_stopping': True, 'k_neighbours': 1, 'n_jobs': 4, 'score': 'f1', 'window_size': 'suss'}
idx:2 {'distance': 'euclidean_distance', 'early_stopping': True, 'k_neighbours': 3, 'n_jobs': 4, 'score': 'roc_auc', 'window_size': 'suss'}
idx:3 {'distance': 'euclidean_distance', 'early_stopping': True, 'k_neighbours': 3, 'n_jobs': 4, 'score': 'f1', 'window_size': 'suss'}
idx:4 {'distance': 'euclidean_distance', 'early_stopping': True, 'k_neighbours': 5, 'n_jobs': 4, 'score': 'roc_auc', 'window_size': 'suss'}
idx:5 {'distance': 'euclidean_distance', 'early_stopping': True, 'k_neighbours': 5, 'n_jobs': 4, 'score': 'f1', 'window_size': 'suss'}
idx:6 {'distance': 'euclidean_distance', 'early_stopping': True, 'k_neighbours': 7, 'n_jobs': 4, 'score': 'roc_auc', 'window_size': 'suss'}
idx:7 {'distance': 'euclidean_dista

In [5]:
#2. splitting
def SplitSaliency(pred):
    # 2. split features in 6 saliency groups
    # vel,acc,jerk,specific,generic
    # specific: other type of feature specific for that saliency
    # generic: other type of feature that describe the whole body movement
    ## generic features: 0,3
    ariel_el=[26,27,28,29,30,31,32,33,34,35,36,37,2,8,14,0,3]
    ariel = UnionCPS(*pred[ariel_el])
   
    strn_el=[38,39,40,41,42,43,44,45,46,47,48,49,1,9,0,3]
    strn = UnionCPS(*pred[strn_el])

    rhel_el=[50,51,52,53,54,55,56,57,58,59,60,61,7,11,16,17,18,21,0,3]
    rhel = UnionCPS(*pred[rhel_el])

    lhel_el=[62,63,64,65,66,67,68,69,70,71,72,73,6,10,16,17,19,20,0,3]
    lhel = UnionCPS(*pred[lhel_el])
    
    rplm_el=[74,75,76,77,78,79,80,81,82,83,84,85,5,13,15,23,25,0,3]
    rplm = UnionCPS(*pred[rplm_el])

    lplm_el=[86,87,88,89,90,91,92,93,94,95,96,97,4,12,15,22,24,0,3]
    lplm = UnionCPS(*pred[lplm_el])

    return [ariel,strn,rhel,lhel,rplm,lplm]

In [6]:
# 3.
def CombineSingleSaliency(saliencies, maj_margin, maj_thre):
    res=[]
    for sal in saliencies:
        res.append(MajorityVoteCP(sal, maj_margin, maj_thre))
    return res
        


In [10]:

#98 per un solo video

# 1. model setting - in: 98 timeseries - out: 98 prediction
# 2. splittare in 6 salienze - in: 98 prediction - out: 98 into 6 list[salienza][feature][elemento singolo] (could be repetition)
# 3xyz. majvote(threshold,margin) - in: 6 list[][][] - out: 6 list[salienza][elemento singolo]
# 3tog. majvote(threshold,margin) - in: 6 list[][][] - out: 6 list[salienza][elemento singolo]
# 4. unione - in: 6 list[][] - out: list[]
# 5. ignorezone(margin)
# 6. delnear(margin)
# 7. dtw(threshold)
def PostFilterTogether(model_id,video_id,param_id,param_len,density,pred,gt, maj_margin, maj_thre, zone_margin, del_margin, dtw_threshold):
    """
        pred: 98 predictions of a single video
    """
    
    print(f"model:{model_id}/20 - video:{video_id}/21 - param:{param_id}/{param_len} - Start                                               ", end='\r')

    #2.
    saliencies_raw = SplitSaliency(pred)
    print(f"model:{model_id}/20 - video:{video_id}/21 - param:{param_id}/{param_len} - Split Saliency completed                            ", end='\r')
    #3.
    saliencies = CombineSingleSaliency(saliencies_raw,maj_margin,maj_thre)
    print(f"model:{model_id}/20 - video:{video_id}/21 - param:{param_id}/{param_len} - Combine Saliency completed                          ", end='\r')
    #4.
    pred_united = UnionCPS(*saliencies)
    print(f"model:{model_id}/20 - video:{video_id}/21 - param:{param_id}/{param_len} - Union completed                                     ", end='\r')
    #5.
    pred_ignored = IgnoreZone(video_id,pred_united,gt,zone_margin)
    print(f"model:{model_id}/20 - video:{video_id}/21 - param:{param_id}/{param_len} - Ignore Zone completed                               ", end='\r')
    #6.
    pred_del = delnear(pred_ignored,del_margin)
    print(f"model:{model_id}/20 - video:{video_id}/21 - param:{param_id}/{param_len} - delnear completed                                   ", end='\r')
    #7.
    pred_final=FPremoverDTW(density,pred_del,dtw_threshold)
    print(f"model:{model_id}/20 - param:{param_id}/{param_len} - video:{video_id}/21 - DTW completed                                       ", end='\r')
    # pred_modelid_videoid_majmargin_majthree_zonemargin_delmargin_dtwthreshold
    return pred_final
    



In [12]:
#leave one video out

param_grid = {
    "maj_margin": [80,100,120],
    "maj_thre": [1,2,3],
    "zone_margin":[80,100,120],
    "del_margin": [80,100,120],
    "dtw_threshold":[80,100,120]
}



# 21*98 = 2058 timeseries
model_res=[]
for model_id,model in enumerate(modelpred): #20 models predictions
    param_value=[]
    for p_idx,params in enumerate(ParameterGrid(param_grid)):
        video=[]
        for i in range(len(TIMESERIES)): #21 videos
            df = dfl[i]
            gt = gtl[i]
            res=PostFilterTogether(model_id,i,p_idx,len(ParameterGrid(param_grid)),df.iloc[:,3],np.array(model[i],dtype="object"),gt,params["maj_margin"],params["maj_thre"],params["zone_margin"],params["del_margin"],params["dtw_threshold"])
            video.append(res)
            """
            i folds servono solo durante lo scoring
            # single fold
            train_set = [j for j in range(len(TIMESERIES)) if j != i]
            test_set = i
            for ts in train_set:
                PostFilterTogether()
            """
        param_value.append(video)
    model_res.append(param_value)

with open("../collection_models_prediction_filtered.pkl", "wb") as f:
    pickle.dump(model_res, f)
    
    #fuori dal for deve calcolare la media
    
"""
versione 1
fisso param_grid
calcolo f1 score per only training fold
prendo il migliore
calcolo per ogni singolo video nel validation

for model in models:
    for param in paramgrid:
        for fold in folds:
            for ts in fold.train:
                f1s_1 = compute single f1score 
                calcolo f1score per singolo ts
            f1s_mean = compute mean f1score mean(f1s_1,f1s_1,...)
            per un intero train_set ritorno lo il f1score medio
        f1s_mean_mean = compute mean of f1s_mean(f1s_mean,f1s_mean,...)
        per ogni train_set ritorno la media = come performa questo param across all folds
    check best f1s_mean_mean
per ogni param diverso, controllo quello che mi massimizza
O(n^3)
"""





'\nversione 1\nfisso param_grid\ncalcolo f1 score per only training fold\nprendo il migliore\ncalcolo per ogni singolo video nel validation\n\nfor model in models:\n    for param in paramgrid:\n        for fold in folds:\n            for ts in fold.train:\n                f1s_1 = compute single f1score \n                calcolo f1score per singolo ts\n            f1s_mean = compute mean f1score mean(f1s_1,f1s_1,...)\n            per un intero train_set ritorno lo il f1score medio\n        f1s_mean_mean = compute mean of f1s_mean(f1s_mean,f1s_mean,...)\n        per ogni train_set ritorno la media = come performa questo param across all folds\n    check best f1s_mean_mean\nper ogni param diverso, controllo quello che mi massimizza\nO(n^3)\n'

<b>Scoring (8)</b>
here we take as input all the filtered results and performs a cross-validation with our predictions to see which hyperparameters for our model+filter is the best and then test it on the validation set

In [3]:
with open("../collection_models_prediction_filtered.pkl", "rb") as f:
    collection_models_prediction_filtered = pickle.load(f)

In [28]:
models=collection_models_prediction_filtered
# model:20 params:243 video:21 len_prediction:26

In [ ]:
# LEAVE ONE VIDEO OUT - LOVO
f1_margin=100
scores=[]
for model_id,params in enumerate(models): #1 model = 243 params <- 20 model
    for p_idx,videos in enumerate(params): # 21 video = 1 param <- 243 params
        score_folds = []
        for video_idx in range(len(TIMESERIES)): # 1 prediction = 1 video <- 1 param = 21 video

            # single fold
            train_set = [i for i in range(len(TIMESERIES)) if i != video_idx]
            test_set = video_idx
            score_singlefold_eachvideo=[]
            for ts in train_set:
                #get score each video for ts
                #compute score
                gt = gtl[ts]
                _,_,f1,_,_,_=f1scoremargin(gt,videos[ts],f1_margin)
                score_singlefold_eachvideo.append(f1)
            #media per training
            score_folds.append(statistics.mean(score_singlefold_eachvideo))
        

        scores.append({"model":model_id, "param":p_idx, "score":statistics.mean(score_folds)})

type_col=["model","param","score"]
resexcel=pd.DataFrame()
resexcel["type"]=type_col
best_models=[]
best_score=0
for i in scores:
    if i["score"] >= best_score:
        best_models.append(i)
best_scores=[]
for best_idx,best in enumerate(best_models):
    best_score=[]
    for video_idx in range(len(TIMESERIES)):
        gt = gtl[video_idx]
        _,_,f1,_,_,_=f1scoremargin(gt,models[best["model"]][best["param"]][video_idx],f1_margin)
        best_score.append(f1)
    best_scores.append({"model":model_id, "param":p_idx, "score":statistics.mean(best_score)})
    resexcel[best_idx]=[model_id,p_idx,statistics.mean(best_score)]

resexcel.to_excel("outputFile/LOVO.xlsx")
print("lovo")
for i in best_scores:
    print(i)






KeyboardInterrupt: 

In [ ]:
# LEAVE ONE DANCER OUT - LODO
CORA=[0,1,2,3,4]
MARIANNE=[5,6,7,8,9,10,11,12,13,14,15,16]
MURIEL=[17,18,19,20]

f1_margin=100
scores=[]
for model_id,params in enumerate(models): #1 model = 243 params <- 20 model
    for p_idx,videos in enumerate(params): # 21 video = 1 param <- 243 params
        score_folds = []
        
        #fold1
        TRAINING_1 = CORA+MARIANNE
        for ts in TRAINING_1:
            #get score each video for ts
            #compute score
            gt = gtl[ts]
            _,_,f1,_,_,_=f1scoremargin(gt,videos[ts],f1_margin)
            score_singlefold_eachvideo.append(f1)
        #media per training
        fold1_score = statistics.mean(score_singlefold_eachvideo)

        #fold2
        TRAINING_2 = CORA+MURIEL
        for ts in TRAINING_2:
            #get score each video for ts
            #compute score
            gt = gtl[ts]
            _,_,f1,_,_,_=f1scoremargin(gt,videos[ts],f1_margin)
            score_singlefold_eachvideo.append(f1)
        #media per training
        fold2_score = statistics.mean(score_singlefold_eachvideo)

        #fold3
        TRAINING_3 = MARIANNE+MURIEL
        for ts in TRAINING_3:
            #get score each video for ts
            #compute score
            gt = gtl[ts]
            _,_,f1,_,_,_=f1scoremargin(gt,videos[ts],f1_margin)
            score_singlefold_eachvideo.append(f1)
        #media per training
        fold3_score = statistics.mean(score_singlefold_eachvideo)

        training_mean_score = statistics.mean[fold1_score,fold2_score,fold3_score]
        scores.append({"model":model_id, "param":p_idx, "score":training_mean_score})

type_col=["model","param","score"]
resexcel=pd.DataFrame()
resexcel["type"]=type_col
best_models=[]
best_score=0
for i in scores:
    if i["score"] >= best_score:
        best_models.append(i)
best_scores=[]
for best_idx,best in enumerate(best_models):

    #fold1
    best_score=[]
    for video_idx in CORA:
        gt = gtl[video_idx]
        _,_,f1,_,_,_=f1scoremargin(gt,models[best["model"]][best["param"]][video_idx],f1_margin)
        best_score.append(f1)
    fold1_score = statistics.mean(best_score)

    #fold2
    best_score=[]
    for video_idx in MARIANNE:
        gt = gtl[video_idx]
        _,_,f1,_,_,_=f1scoremargin(gt,models[best["model"]][best["param"]][video_idx],f1_margin)
        best_score.append(f1)
    fold2_score = statistics.mean(best_score)

    #fold3
    best_score=[]
    for video_idx in MURIEL:
        gt = gtl[video_idx]
        _,_,f1,_,_,_=f1scoremargin(gt,models[best["model"]][best["param"]][video_idx],f1_margin)
        best_score.append(f1)
    fold3_score = statistics.mean(best_score)

    val_mean_score = statistics.mean([fold1_score,fold2_score,fold3_score])
   
    best_scores.append({"model":model_id, "param":p_idx, "score":val_mean_score})
    resexcel[best_idx]=[model_id,p_idx,val_mean_score]

resexcel.to_excel("outputFile/LODO.xlsx")
print("lodo")
for i in best_scores:
    print(i)
